# AS05: Implementação de Assistente Conversacional Baseado em LLM

Nome: Henrique Mendonça Castelar Campos

Matrícula: 618557

## GPT

Instalando requisitos

In [9]:
!pip install -q torch transformers accelerate

Fazendo login no Hugging Face Hub

In [2]:
from huggingface_hub import login

huggingfacehub_api_token = ""
login(huggingfacehub_api_token)

Criando modelo

Como a API da Open AI não está mais disponível gratuitamente, resolvi utilizar um modelo local

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

model_name = "Qwen/Qwen2.5-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto")
chatbot = pipeline("text-generation", model=model, tokenizer=tokenizer)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/7.23k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/684 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/138 [00:00<?, ?B/s]

Device set to use cpu


Testando o modelo

In [4]:
chatbot("How much wood would a woodchuck chuck if a woodchuck could chuck wood?")

[{'generated_text': 'How much wood would a woodchuck chuck if a woodchuck could chuck wood? If a woodchuck could chuck wood, it would chuck at the rate of one piece of wood for every two days of life. If a woodchuck lives for 400 days, how many pieces of wood would it chuck?\n\nTo determine how many pieces of wood a woodchuck would chuck if it lives for 400 days, we can use the given rate: one piece of wood for every two days of life. We can express this relationship mathematically:\n\n\\[ \\text{Number of pieces of wood} = \\frac{\\text{ Total days of life}}{\\text{ Days of life per piece of wood}} \\]\n\nHere, the total days of life is 400 days, and the days of life per piece of wood is 2 days. Plugging in these values, we get:\n\n\\[ \\text{Number of pieces of wood} = \\frac{400}{2} = 200 \\]\n\nTherefore, if a woodchuck lives for 400 days, it would chuck \\(\\boxed{200}\\) pieces of wood.'}]

In [5]:
chatbot("Qual é a capital do Canadá?")

[{'generated_text': 'Qual é a capital do Canadá? | Quero saber\nQuando perguntamos qual é a capital do Canadá, a resposta está em Montreal. A cidade é sede das autoridades federais, o que significa que os ministérios e os governos federais se concentram lá.\nO governo federal também tem sede em Ottawa, mas o centro da cidade é Montreal. Montreal é considerada a capital do Canadá, mas não é oficialmente a capital. Ottawa é a sede do governo federal, e Montreal é a sede da administração federal.\nA cidade de Montreal é o centro financeiro do Canadá e tem uma economia bem desenvolvida. A cidade é também famosa por seu clima e temperaturas estacionadas, permitindo que as pessoas vivam lá o ano todo.\nMontreal é uma cidade grande e cosmopolita, e ela é a maior cidade da província de Quebec, que é uma das províncias do Canadá. Montreal é uma cidade de classe mundial, e ela é famosa por sua cultura e história.\nA cultura de Montreal é muito diversa e influenciada por diversos países e cultura

## Pinecone

Instalando requisitos

In [8]:
!pip install -q pinecone

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.3/516.3 kB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.1/239.1 kB 8.4 MB/s eta 0:00:00


Criando índice PineCone

In [33]:
from pinecone import Pinecone

pinecone_api_key = ""
pinecone = Pinecone(pinecone_api_key)

Obtendo dataset

In [34]:
from pinecone_datasets import load_dataset

dataset = load_dataset('quora_all-MiniLM-L6-bm25')
# we drop sparse_values as they are not needed for this example
dataset.documents.drop(['metadata'], axis=1, inplace=True)
dataset.documents.rename(columns={'blob': 'metadata'}, inplace=True)
# we will use 88K rows of the dataset between rows 240K -> 320K
dataset.documents.drop(dataset.documents.index[320_000:], inplace=True)
dataset.documents.drop(dataset.documents.index[:240_000], inplace=True)
dataset.head()

Loading documents parquet files:   0%|          | 0/10 [00:00<?, ?it/s]

,id,values,sparse_values,metadata
240000,515997,"[-0.00531694, 0.06937869, -0.0092854, 0.003286...","{'indices': [845, 1657, 13677, 20780, 27058, 2...","{'text': ' Why is a ""law of sciences"" importan..."
240001,515998,"[-0.09243751, 0.065432355, -0.06946959, 0.0669...","{'indices': [2110, 6324, 9754, 13677, 15207, 2...",{'text': ' Is it possible to format a BitLocke...
240002,515999,"[-0.021924071, 0.032280188, -0.020190848, 0.07...","{'indices': [2110, 4949, 23579, 23758, 27058, ...",{'text': ' Can formatting a hard drive stress ...
240003,516000,"[-0.120020054, 0.024080949, 0.10693012, -0.018...","{'indices': [22014, 24734, 24773, 25791, 25991...",{'text': ' Are the new Samsung Galaxy J7 and J...
240004,516001,"[-0.095293395, -0.048446465, -0.017618902, -0....","{'indices': [307, 2110, 5785, 12969, 12971, 13...",{'text': ' I just watched an add for Indonesia...


Criando indice

In [39]:
nome_indice = 'llama-text-embed-v2-index'

pinecone_index = pinecone.Index(nome_indice)

dados = [
    {"id": "vec1", "text": "Apple is a popular fruit known for its sweetness and crisp texture."},
    {"id": "vec2", "text": "The tech company Apple is known for its innovative products like the iPhone."},
    {"id": "vec3", "text": "Many people enjoy eating apples as a healthy snack."},
    {"id": "vec4", "text": "Apple Inc. has revolutionized the tech industry with its sleek designs and user-friendly interfaces."},
    {"id": "vec5", "text": "An apple a day keeps the doctor away, as the saying goes."},
    {"id": "vec6", "text": "Apple Computer Company was founded on April 1, 1976, by Steve Jobs, Steve Wozniak, and Ronald Wayne as a partnership."}
]

pinecone_index.upsert_records(
    namespace='example-namespace',
    records=dados
)

Realizando consulta

In [46]:
consulta = "Tell me about the tech company known as Apple."

resultados = pinecone_index.search(
    namespace='example-namespace',
    query={
        "inputs": {"text": consulta},
        "top_k": 3
    }
)

resultados

{'result': {'hits': [{'_id': 'vec2',
                      '_score': 0.6304798722267151,
                      'fields': {'text': 'The tech company Apple is known for '
                                         'its innovative products like the '
                                         'iPhone.'}},
                     {'_id': 'vec4',
                      '_score': 0.5368806719779968,
                      'fields': {'text': 'Apple Inc. has revolutionized the '
                                         'tech industry with its sleek designs '
                                         'and user-friendly interfaces.'}},
                     {'_id': 'vec6',
                      '_score': 0.35400307178497314,
                      'fields': {'text': 'Apple Computer Company was founded '
                                         'on April 1, 1976, by Steve Jobs, '
                                         'Steve Wozniak, and Ronald Wayne as a '
                                         'partner

## LangChain

Instalando requisitos

In [47]:
!pip install langchain

Criando template

In [9]:
from langchain.prompts.chat import ChatPromptTemplate

class ParserSeparadoPorVirrgula(ChatPromptTemplate):
    def parse(self, text: str):
        return text.strip().split(", ")

template = """Você é um assistente que gera uma listas separadas por vírgula.
Um usuário passa uma categoria e você gera 5 objetos nessa categoria em uma lista separada por vírgula.
Apenas retorna a lista separada por vírgula, e nada mais."""
template_humano = "{text}"

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", template),
    ("human", template_humano)
])

In [14]:
prompt = chat_prompt.format(text="comida")
response = chatbot(prompt)
response

[{'generated_text': 'System: Você é um assistente que gera uma listas separadas por vírgula.\nUm usuário passa uma categoria e você gera 5 objetos nessa categoria em uma lista separada por vírgula.\nApenas retorna a lista separada por vírgula, e nada mais.\nHuman: comida vegetariana\nfrutas\nfrutas\nbebidas\nbebidas'}]